In [4]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random as rd

In [5]:
def elegir_columnas_random(lista_columnas):
    cantidad = rd.randint(0, len(list(set_entrenamiento)))
    for x in range(cantidad):
        c = rd.choice(lista_columnas)
        if c == 'person' or c == 'label':
            continue
        lista_columnas.remove(c)
        if len(lista_columnas) < 10:
            break
    return lista_columnas

def dar_columnas_restantes(set_e,columnas):
    columnas_totales = list(set_e)
    for c in columnas_totales:
        if c in columnas:
            columnas_totales.remove(c)
    return columnas_totales

In [11]:
training_labels = pd.read_csv("set_entrenamiento.csv", low_memory=False)[['person','label']]

In [17]:
resultados = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
rd.seed(1) #tomando siempre la misma semilla nuestro modulo "random" ya no es random.
for x in range(100):
    training_labels = pd.read_csv("set_entrenamiento.csv", low_memory=False)[['person','label']]
    labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
    events_data = pd.read_csv("data_set.csv", low_memory = False).drop(columns = {'days_since_last_session'})
    set_entrenamiento = pd.merge(events_data, training_labels, on ='person', how = 'inner')
    columnas = elegir_columnas_random(list(set_entrenamiento))
    set_entrenamiento = set_entrenamiento[columnas]
    #esta horrible esto, cambiar
    if 'label' in columnas:
        columnas.remove('label')
    events_data = events_data[columnas]
    set_entrenamiento = set_entrenamiento.drop(columns = ['person'])
    X = set_entrenamiento.drop(columns = ['label'])
    y = set_entrenamiento["label"]
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123)
    data_dmatrix = xgb.DMatrix(data=X,label=y)
    xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 6, alpha = 5, n_estimators = 20)
    xg_reg.fit(x_train,y_train)
    preds = xg_reg.predict(x_test)
    preds = preds.tolist()
    df = pd.DataFrame(preds)
    media = df[0].mean()
    std = df[0].std()
    df = df[0].map(lambda x: 1 if ((x-media)/std) > 0 else 0)
    set_test = pd.merge(events_data,labels_predict,on = "person", how = "right")
    personas = set_test["person"]
    set_test = set_test.drop(columns=["person"])
    set_test = set_test.fillna(0)
    predicts_kaggel = xg_reg.predict(set_test)
    predicts_kaggel = predicts_kaggel.tolist()
    df_kaggel = pd.DataFrame(predicts_kaggel)
    media = df_kaggel[0].mean()
    std = df_kaggel[0].std()
    df_kaggel = df_kaggel[0].map(lambda x: 1 if ((x-media)/std) > 0 else 0);
    df_kaggel = df_kaggel.to_frame()
    df_kaggel["person"] = personas
    df_kaggel.rename(columns = {0: 'label'},inplace = True)
    df_kaggel = df_kaggel[['person','label']]
    resultados = pd.merge(resultados,df_kaggel,on='person',how='inner')

### Vemos resultados y pasamos a un csv

In [18]:
resultados.head()

,person,label_x,label_y,label_x,label_y,label_x,label_y,label_x,label_y,label_x,...,label_x,label_y,label_x,label_y,label_x,label_y,label_x,label_y,label_x,label_y
0,4886f805,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,0,0
1,0297fc1e,1,1,0,1,1,1,1,1,1,...,1,1,0,1,1,1,1,0,1,1
2,2d681dd8,0,0,0,0,1,0,1,0,0,...,1,1,1,0,0,0,0,1,0,1
3,cccea85e,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,4c8a8b93,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [20]:
resultados.to_csv('Resultados.csv',index=False)